In [1]:
import pandas as pd
import numpy as np
import json
import joblib
import gensim
import re
import pickle
import warnings
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

unable to import 'smart_open.gcs', disabling that module


In [2]:
# Let's import our data
data = pd.read_csv('./processedIndiafinal.csv')

In [3]:
flairs = ['AskIndia', 'Business/Finance',
        'CAA-NRC-NPR', 'Coronavirus',
         'Food',  'Non-Political',
         'Policy/Economy',  'Politics',
          'Scheduled', 'Science/Technology',
          'Sports', 'TIL', 'Unverified', 'na', 'Photography']

In [5]:
# Let's split our data into train and test
# We will use 75:25 rule, 75% for train and 25% for test
# Let's create the function for it
# we will use train_test_split function for it which comes with sklearn
# setting random state to 10, we can set it to any fixed value

def train_test(X,y): 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 10)
    print("Naive Bayes Classifier ")
    naivebayes_classifier(X_train, X_test, y_train, y_test)
    print("Linear Support Vector Machine ")
    lsvm(X_train, X_test, y_train, y_test)
    print("Logistic Regression ")
    logistic_regression(X_train, X_test, y_train, y_test)
    print("Random Forest ")
    random_forest(X_train, X_test, y_train, y_test)

In [21]:
# Let's create a function for each classifier

# Naive Bayes
def naivebayes_classifier(X_train, X_test, y_train, y_test):
    nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
    nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)
    print("Accuracy: "+str(accuracy_score(y_pred,y_test)))

# Linear Support Vector Machine
def lsvm(X_train, X_test, y_train, y_test):
    sgd = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=10, max_iter=5, tol=None)),
                 ])
    sgd.fit(X_train, y_train)
    y_pred = sgd.predict(X_test)
    print("Accuracy: "+str(accuracy_score(y_pred,y_test)))

# Logistic Regression
def logistic_regression(X_train, X_test, y_train, y_test):
    logreg = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e30)),
                 ])
    logreg.fit(X_train, y_train)
    y_pred = logreg.predict(X_test)
    print("Accuracy: "+str(accuracy_score(y_pred,y_test)))
    
# Random Forest
def random_forest(X_train, X_test, y_train, y_test):
    ranfor = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 10)),
                 ])
    ranfor.fit(X_train, y_train)
    y_pred = ranfor.predict(X_test)
    print("Accuracy: "+str(accuracy_score(y_pred,y_test)))


In [ ]:
warnings.filterwarnings('ignore')
print("Flair Detection using Title as a Feature:")
print("---------------------------------------")
train_test(data.title, data.flair)

Flair Detection using Title as a Feature:
---------------------------------------
Naive Bayes Classifier 
Accuracy: 0.9668624161073825
Linear Support Vector Machine 
Accuracy: 0.9739932885906041
Logistic Regression 
Accuracy: 0.9979026845637584
Random Forest 


In [18]:
print("Flair Detection using Body as a Feature:")
print("---------------------------------------")
train_test(data.body, data.flair)

Flair Detection using Body as a Feature:
---------------------------------------
Naive Bayes Classifier 
Accuracy: 0.5880872483221476
                    precision    recall  f1-score   support

          AskIndia       0.00      0.00      0.00        19

         micro avg       0.00      0.00      0.00        19
         macro avg       0.00      0.00      0.00        19
      weighted avg       0.00      0.00      0.00        19

Linear Support Vector Machine 
Accuracy: 0.3921979865771812
                    precision    recall  f1-score   support

          AskIndia       1.00      0.37      0.54        19

         micro avg       1.00      0.37      0.54        19
         macro avg       1.00      0.37      0.54        19
      weighted avg       1.00      0.37      0.54        19

Logistic Regression 
Accuracy: 0.5931208053691275
                    precision    recall  f1-score   support

          AskIndia       1.00      0.99      0.99       342
  Business/Finance       1.00

In [17]:
print("Flair Detection using Title+Body+URL as a Feature:")
print("---------------------------------------")
train_test(data.feature_tbu, data.flair)

Flair Detection using Title+Body+URL as a Feature:
---------------------------------------
Naive Bayes Classifier 
Accuracy: 0.9442114093959731
                    precision    recall  f1-score   support

          AskIndia       0.00      0.00      0.00        19

         micro avg       0.00      0.00      0.00        19
         macro avg       0.00      0.00      0.00        19
      weighted avg       0.00      0.00      0.00        19

Linear Support Vector Machine 
Accuracy: 0.988255033557047
                    precision    recall  f1-score   support

          AskIndia       1.00      0.95      0.97        19

         micro avg       1.00      0.95      0.97        19
         macro avg       1.00      0.95      0.97        19
      weighted avg       1.00      0.95      0.97        19

Logistic Regression 
Accuracy: 0.9979026845637584
                    precision    recall  f1-score   support

          AskIndia       1.00      1.00      1.00       342
  Business/Finance  

### After detailed analysis of all the metrics!
### Combined feature of Title, Body and URL performed better than others.
### Logistic Regression performed better than other algorithms!

In [19]:
X=data.title
y=data.flair
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 10)
logreg = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e30)),
                 ])
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [20]:
# Let's Dump our model
pickle.dump(logreg, open('final_model.pkl','wb'))